In [1]:
import json
import pandas
import utils
import itertools
from itertools import combinations_with_replacement
import copy

# Get the character_stats, weapon stats and artifact stats that you want

In [2]:
character = "characters/soldier11.json"
weapon = "weapons/starlight_engine_r5.json"
disk = "disks_setup/disk_options.json"
# excel_save_name = "results/ellen_sig.xlsx"

# other constants
constants = "constants.json"
target_path = "enemies/tyrfing.json"

substat_points = 34

damage_combo = ["ultimate"]

# Real calculations begin

In [3]:
# load character, weapon, disk, and constants
character_stats, base_motion_values, weapon_stats, other_stats = utils.get_stats(character, weapon, disk)
disk_stats = utils.evaluate_disk_stat(disk_options = other_stats)
total_stats, target_stats = utils.get_total_stats(
    character_stats,
    weapon_stats,
    disk_stats,
    target_path
)
new_mv = utils.motion_to_damage(base_motion_values, total_stats, target_stats)
final_damage = utils.combo_to_damage(damage_combo, new_mv)
print(final_damage)

129239.3895631893


In [4]:
character_stats, base_motion_values, weapon_stats, other_stats = utils.get_stats(character, weapon, disk)

In [5]:
def disk_possibilities(
    character_stats,
    base_motion_values,
    weapon_stats,
    other_stats,
    substat_points = 2,
    max_per_stat = 4,
):
    best_damage = 0
    best_allocation = None
    # Count occurrences of each substat in the allocation
    valid_substat_combinations = utils.generate_valid_combinations(
        substat_points,
        max_per_stat,
    )
    disk_4_combo, disk_5_combo, disk_6_combo = utils.generate_main_disk_options()
    for disk_4_allocation in disk_4_combo:
        for disk_5_allocation in disk_5_combo:
            for disk_6_allocation in disk_6_combo:
                main_stat_dict = {
                    "disk_4": disk_4_allocation,
                    "disk_5": disk_5_allocation,
                    "disk_6": disk_6_allocation,
                }
                for substat_allocations in valid_substat_combinations:
                    # make a copy of every dictionary in case we directly modify the dictionary
                    character_stats_copy = copy.deepcopy(character_stats)
                    base_motion_values_copy = copy.deepcopy(base_motion_values)
                    weapon_stats_copy = copy.deepcopy(weapon_stats)
                    other_stats_copy = copy.deepcopy(other_stats)
                    
                    # make a copy of every dictionary in case we directly modify the dictionary
                    disk_stats = utils.evaluate_disk_stat(
                        disk_options = other_stats_copy,
                        main_stat_dict = main_stat_dict,
                        sub_stat_dict = substat_allocations
                    )
                    total_stats, target_stats = utils.get_total_stats(
                        character_stats_copy,
                        weapon_stats_copy,
                        disk_stats,
                        target_path
                    )
                    new_mv = utils.motion_to_damage(base_motion_values_copy, total_stats, target_stats)
                    final_damage = utils.combo_to_damage(damage_combo, new_mv)
                    if final_damage > best_damage:
                        best_damage = final_damage
                        best_main_stats = main_stat_dict
                        best_allocation = substat_allocations
                        best_total_stats = total_stats

    print(best_damage)
    print(best_main_stats)
    print(best_allocation)
    print(best_total_stats)


disk_possibilities(
    character_stats,
    base_motion_values,
    weapon_stats,
    other_stats,
    substat_points = 34,
    max_per_stat=24
)

135578.2683865931
{'disk_4': 'CRIT_Rate', 'disk_5': 'Elemental_Damage', 'disk_6': 'ATK_%'}
{'Flat_ATK': 0, 'ATK_%': 0, 'CRIT_Rate': 13, 'CRIT_DMG': 21, 'Flat_PEN': 0}
{'Level': 60, 'Level_Coefficient': 281, 'DEF_Reduction': 0, 'Stun_Bonus': 0, 'Total_HP': 9872.3, 'Total_DEF': 797.3, 'Total_ATK': 3115.868332, 'CRIT_Rate': 0.746, 'CRIT_DMG': 1.508, 'PEN_Ratio': 0, 'Impact': 93, 'Anomaly_Mastery': 94, 'Anomaly_Proficiency': 93, 'Energy_Regen': 1.2, 'Buff_Level_Multiplier': 1.9970999999999999, 'DMG%_Multiplier': 1.625, 'DMG_Taken_Multiplier': 1}
